# 빅데이터 분석기사 2유형 템플릿

## 회귀모델

In [3]:
# 모듈 불러오기
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder          # 인코딩(분류형)
from sklearn.preprocessing import StandardScaler        # 인코딩(수치형)
from sklearn.model_selection import train_test_split    # 데이터 분할

# 분류모델
from sklearn.ensemble import RandomForestClassifier     # 분류모델, 랜덤포레스트
from xgboost import XGBClassifier                       # 분류모델, XGboost
from sklearn.metrics import roc_auc_score               # 평가지표(문제에 따라)


In [ ]:
# 1. 데이터 불러오기
# 문제에서 제시
X_train = pd.read_csv('~~~')
X_test = pd.read_csv('~~~') 
Y_train = pd.read_csv('~~~')


In [5]:
# 2. 데이터 전처리
# 변수 나누기
# 변수 자료타입에 따라서 num과 cat에 나눠넣기
num_cols = []       #수치형 변수
cat_cols = []       #범주형 변수
y_cols = []         #예측대상(종속변수)

# tip) 컬럼 리스트 가져오기 (컬럼명 직접 적지 마세요)
# X_train.columns 로 출력


In [ ]:
# 결측치 갯수 확인
print(X_train.isnull().sum())
print(Y_train.isnull().sum())

# variable.info(), describe() 등으로도 확인 가능
#결측치가 있을 경우
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy = 'most_frequent')   #최빈값 채우기로 수치형 범주형 모두 가능
X_train[cat_cols + num_cols] = imp.fit_transform(X_train[cat_cols + num_cols])
X_test[cat_cols + num_cols] = imp.fit_transform(X_test[cat_cols + num_cols])
#X_train[['Var1']] = imp.fit_transform(X_train[['Var1']])


In [ ]:
# 3. 데이터 모형 구축
# - 데이터 분할
X_tr, X_val, y_tr, y_val = train_test_split(X_train[cat_cols + num_cols],
                                            y_train[y_col].values.ravel(),    # pd.Series -> nd.array
                                            test_size=0.3,
                                            stratify= y_train[y_col].values.ravel(), # 분류모형, test set과 validation set의 분류별 비율을 고정함
                                            random_state=42)   # 재현을 위해서(선택)


In [ ]:
# num_cols 데이터 스케일링 
sc = StandardScaler()
sc.fit(X_tr[num_cols])   # 누수방지를 위해 fit은 X_tr로만 수행(기준을 잡아주기) 후, 나머지에 대해 스케일링하기
X_tr[num_cols]   = sc.transform(X_tr[num_cols])
y_tr[num_cols]   = sc.transform(y_tr[num_cols])
X_test[num_cols] = sc.transform(X_test[num_cols])                   


< variable.values.ravel() 설명 >
1. values: 이는 pandas 데이터프레임(DataFrame) 또는 시리즈(Series)에서 실제 값들을 추출하는 메소드입니다. 이를 통해 데이터프레임이나 시리즈에 저장된 값들을 NumPy 배열 형태로 반환합니다.
2. ravel(): 이는 다차원 배열을 1차원 배열로 평탄화(flatten)하는 함수입니다. 만약 데이터가 이미 1차원 배열이라면 변화 없이 그대로 반환됩니다. 일반적으로 머신러닝에서는 입력 데이터가 1차원 배열 형태로 사용되기 때문에 이러한 평탄화 과정이 필요합니다.

In [ ]:
# 데이터 스케일링 
## num_cols
sc = StandardScaler()
sc.fit(X_tr[num_cols])   # 누수방지를 위해 fit은 X_tr로만 수행(기준을 잡아주기) 후, 나머지에 대해 스케일링하기
X_tr[num_cols]   = sc.transform(X_tr[num_cols])
y_tr[num_cols]   = sc.transform(y_tr[num_cols])
X_test[num_cols] = sc.transform(X_test[num_cols])


## cat_cols
X = pd.concat([X_train[cat_cols], X_test[cat_cols]])  

for col in cat_cols:     # 라벨인코더는 각 피쳐별로 수행해야 함
    le = LabelEncoder()
    le.fit(X[col])       # 누수 유의
    X_tr[col]   = sc.transform(X_tr[col])
    y_tr[col]   = sc.transform(y_tr[col])
    X_test[col] = sc.transform(X_test[col])

    # 변수 클래스 확인방법(not 필수)
    print(col)
    print(le.classes_, '\n')


In [ ]:
# 분류모델 학습
m1 = RandomForestClassifier()
m1.fit(X_tr, y_tr)
pred1 = m1.predict_proba(X_val)[:, 1]  #predict_proba 시 출력은 [0일 확률, 1일 확률]로 구성됨 # roc-auc 스코어를 쓰기 때문에, thread-hold에 영향을 받지 않음.

m2 = XGBClassifier()
m2.fit(X_tr, y_tr)
pred2 = m2.predict_proba(X_val)[:, 1]


In [ ]:
# 4. 모형평가
# 분류모델 평가
score1 = roc_auc_score(y_val, pred1)
score2 = roc_auc_score(y_val, pred1)

print(score1, score2)   #roc는 점수가 높은 모델을 선정

#최종 모델 선정
best_model = m1   # m1 or m2


In [ ]:
# 5. 결과 생성
# 분류모델의 경우, 채점 평가 지표에 따라 예측값인지, 예측 확률인지를 결정함

# 예측 **확률** 제출  roc_auc_score 인 경우?
  # pred = best_model.predict_proba(X_test[cat_cols + num_cols])[:,1]

# 예측 **결과** 제출   accuracy, F1 등 confusion matrix 기반 성능평가시
  # pred = best_model.predict(X_test[cat_cols + num_cols])[:,1]

#결과제출
# 답안 제출 참고
# 아래 코드는 예시이며 변수명 등 개인별로 변경하여 활용
result = pd.DataFrame('pred': pred)
result.to_csv("result.csv", index=False)


## 분류모델

In [13]:
# 0. 모듈 임포트

# 공통모듈
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 회귀모델
from sklearn.ensemble import RandomForestRegressor   # 회귀모델, 랜덤포레스트
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error  #평가지표 MSE


In [ ]:
# 1. 데이터 불러오기
# - 문제에 나온 경로 그대로
X_train = pd.read_csv()
y_train = pd.read_csv()
X_test = pd.read_csv()


In [ ]:
# 2. 데이터 전처리
# 변수 나누기
# 변수 자료타입에 따라서 num과 cat에 나눠넣기
num_cols = []       #수치형 변수
cat_cols = []       #범주형 변수
y_cols = []         #예측대상(종속변수)

# tip) 컬럼 리스트 가져오기 (컬럼명 직접 적지 마세요)
# X_train.columns 로 출력

In [ ]:
# 결측치 갯수 확인
print(X_train.isnull().sum())
print(Y_train.isnull().sum())

# variable.info(), describe() 등으로도 확인 가능
#결측치가 있을 경우
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy = 'most_frequent')   #최빈값 채우기로 수치형 범주형 모두 가능
X_train[cat_cols + num_cols] = imp.fit_transform(X_train[cat_cols + num_cols])
X_test[cat_cols + num_cols] = imp.fit_transform(X_test[cat_cols + num_cols])
#X_train[['Var1']] = imp.fit_transform(X_train[['Var1']])


In [ ]:
# 3. 데이터 모형 구축
# - 데이터 분할
X_tr, X_val, y_tr, y_val = train_test_split(X_train[cat_cols + num_cols],     # 독립변수
                                            y_train[y_col].values.ravel(),    # 종속변수, pd.Series -> nd.array
                                            test_size=0.3,                    # 7:3 분할
                                            random_state=42)   # 재현을 위해서


< variable.values.ravel() 설명 >
1. values: 이는 pandas 데이터프레임(DataFrame) 또는 시리즈(Series)에서 실제 값들을 추출하는 메소드입니다. 이를 통해 데이터프레임이나 시리즈에 저장된 값들을 NumPy 배열 형태로 반환합니다.
2. ravel(): 이는 다차원 배열을 1차원 배열로 평탄화(flatten)하는 함수입니다. 만약 데이터가 이미 1차원 배열이라면 변화 없이 그대로 반환됩니다. 일반적으로 머신러닝에서는 입력 데이터가 1차원 배열 형태로 사용되기 때문에 이러한 평탄화 과정이 필요합니다.

In [ ]:
# 데이터 스케일링 
## num_cols
sc = StandardScaler()
sc.fit(X_tr[num_cols])   # 누수방지를 위해 fit은 X_tr로만 수행(기준을 잡아주기) 후, 나머지에 대해 스케일링하기
X_tr[num_cols]   = sc.transform(X_tr[num_cols])
y_tr[num_cols]   = sc.transform(y_tr[num_cols])
X_test[num_cols] = sc.transform(X_test[num_cols])


## cat_cols
X = pd.concat([X_train[cat_cols], X_test[cat_cols]])  

for col in cat_cols:     # 라벨인코더는 각 피쳐별로 수행해야 함
    le = LabelEncoder()
    le.fit(X[col])       # 누수 유의
    X_tr[col]   = sc.transform(X_tr[col])
    y_tr[col]   = sc.transform(y_tr[col])
    X_test[col] = sc.transform(X_test[col])

    # 변수 클래스 확인방법(not 필수)
    print(col)
    print(le.classes_, '\n')


In [ ]:
# 분류모델 학습
## 회귀모델에는 proba가 없음.
## predict 시 출력은 예측값을 직접 출력
m1 = RandomForestRegressor()
m1.fit(X_tr, y_tr)
pred1 = m1.predict(X_val)[:, 1]  


m2 = XGBRegressor()
m2.fit(X_tr, y_tr)
pred2 = m2.predict(X_val)[:, 1]


In [ ]:
# 4. 모형평가
# 분류모델 평가
score1 = mean_squared_error(y_val, pred1)
score2 = mean_squared_error(y_val, pred1)

print(score1, score2)   # mse는 적을수록 정확한 예측, 점수가 적은 모델을 선정

#최종 모델 선정
best_model = m1   # m1 or m2


In [ ]:
# 5. 결과 생성
# 분류모델의 경우, 채점 평가 지표에 따라 예측값인지, 예측 확률인지를 결정함

# 예측 **확률** 제출  roc_auc_score 인 경우?
  # pred = best_model.predict_proba(X_test[cat_cols + num_cols])[:,1]

# 예측 **결과** 제출   accuracy, F1 등 confusion matrix 기반 성능평가시
  # pred = best_model.predict(X_test[cat_cols + num_cols])[:,1]

#결과제출
# 답안 제출 참고
# 아래 코드는 예시이며 변수명 등 개인별로 변경하여 활용
result = pd.DataFrame('pred': pred)
result.to_csv("result.csv", index=False)
